In [1]:
!pip install langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 1.7 MB/s eta 0:00:00


In [2]:
import json
import pickle
import pandas as pd
import numpy as np
from getpass import getpass
from tqdm import tqdm
from typing import List

from langchain_groq import ChatGroq
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

In [3]:
GROQ_API_KEY = getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


In [4]:
llama3_70b_temp0 = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192",
    api_key=GROQ_API_KEY)

# Test
llama3_70b_temp0.invoke("What is the capital of France?")

AIMessage(content='The capital of France is Paris.', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 17, 'total_tokens': 25, 'completion_time': 0.022857143, 'prompt_time': 0.004959961, 'queue_time': None, 'total_time': 0.027817104}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_2f30b0b571', 'finish_reason': 'stop', 'logprobs': None}, id='run-cf3a7624-f4f7-4e68-996b-a6b4824ad9bd-0')

In [5]:
df = pd.read_csv("/content/dataset_concurso_rag.csv")
df.columns, len(df)

(Index(['pergunta', 'resposta', 'comentarios', 'numero_na_captura', 'id',
        'area', 'ano', 'banca', 'prova', 'duvidou_do_gabarito', 'enunciado',
        'assertivas', 'alternativas', 'classificacao',
        'artigos_nos_comentarios', 'artigos_completos', 'artigo_inventado',
        'topicos_para_busca', 'artigos_candidatos', 'frase_assunto_da_questao',
        'rag_art_inventado_full', 'rag_art_inventado_full_sem_art',
        'rag_assunto_full', 'rag_topicos_full', 'rag_artigos_candidatos',
        'rag_artigos_candidatos_grupos'],
       dtype='object'),
 37)

In [73]:
contexto_df = pd.read_csv("/content/dataset2_Concursos_Contexto_todos.csv")
contexto_df

,Contexto
0,"Art 1214,Art 1975,Art 206,Art 1974,1826,Art 19..."
1,"Art 1880,Art 1879,Art 1867,Art 1868"
2,"Art 1858,Art 1860,Art 1880,Art 1876,Art 1859,A..."
3,"Art 1860,Art 1861,Art 1802,Art 1900,Art 1857,A..."
4,"Art 1723,Art 1521"
5,Art 276
6,"Art 1647,Art 1580,Art 1648,Art 1651,Art 1597,A..."
7,Art 270
8,Art 1813
9,"Art 1860,Art 1858,Art 1857,Art 1845,Art 1786"


In [74]:
with open("/content/codigo_civil.json", "r") as f:
    codigo = json.load(f)

def collect_artigo_and_header(data):
    result = {'articles': [], 'contexts': []}

    def traverse(data):
        if isinstance(data, dict):
            for key, value in data.items():
                if key == 'artigos' and value:
                    result['articles'].append(value)
                if key == 'header' and value:
                    result['contexts'].append(value)
                # Recursive call to handle nested dictionaries
                if isinstance(value, dict):
                    traverse(value)
                elif isinstance(value, list):
                    traverse(value)
        elif isinstance(data, list):
            for item in data:
                # Recursive call to handle lists of dictionaries
                traverse(item)

    traverse(data)
    return result

artigos = collect_artigo_and_header(codigo)['articles']

In [75]:
def get_article(art_id, artigos=artigos):
    for art_group in artigos:
        for artigo in art_group:
            if art_id.replace('.', '') in artigo[:10].replace('.', ''): # limit to get article name in init
                return artigo.replace('\n\n', '').strip()

def convert_strlist_to_liststr(strlist):
    strlist = strlist.replace("'", "").replace('Art', 'Art.').replace('Art. 1815-A', '') # remove '
    liststr = strlist.split(",") # split elements
    return liststr

In [76]:
contexto_df['OldContexto'] = contexto_df['Contexto']
contexto_df['Contexto'] = contexto_df['Contexto'].apply(convert_strlist_to_liststr)
contexto_df

,Contexto,OldContexto
0,"[Art. 1214, Art. 1975, Art. 206, Art. 1974, 18...","Art 1214,Art 1975,Art 206,Art 1974,1826,Art 19..."
1,"[Art. 1880, Art. 1879, Art. 1867, Art. 1868]","Art 1880,Art 1879,Art 1867,Art 1868"
2,"[Art. 1858, Art. 1860, Art. 1880, Art. 1876, A...","Art 1858,Art 1860,Art 1880,Art 1876,Art 1859,A..."
3,"[Art. 1860, Art. 1861, Art. 1802, Art. 1900, A...","Art 1860,Art 1861,Art 1802,Art 1900,Art 1857,A..."
4,"[Art. 1723, Art. 1521]","Art 1723,Art 1521"
5,[Art. 276],Art 276
6,"[Art. 1647, Art. 1580, Art. 1648, Art. 1651, A...","Art 1647,Art 1580,Art 1648,Art 1651,Art 1597,A..."
7,[Art. 270],Art 270
8,[Art. 1813],Art 1813
9,"[Art. 1860, Art. 1858, Art. 1857, Art. 1845, A...","Art 1860,Art 1858,Art 1857,Art 1845,Art 1786"


In [71]:
get_article('Art. 1815')

'Art. 1.815. A exclusão do herdeiro ou legatário, em qualquer desses casos de indignidade, será declarada por sentença.Parágrafo único. O direito de demandar a exclusão do herdeiro ou legatário extingue-se em quatro anos, contados da abertura da sucessão.§ 1 o O direito de demandar a exclusão do herdeiro ou legatário extingue-se em quatro anos, contados da abertura da sucessão.     (Redação dada pela Lei nº 13.532, de 2017)§ 2 o Na hipótese do inciso I do art. 1.814, o Ministério Público tem legitimidade para demandar a exclusão do herdeiro ou legatário.     (Incluído pela Lei nº 13.532, de 2017)  \nArt. 1.815-A. Em qualquer dos casos de indignidade previstos no art. 1.814, o trânsito em julgado da sentença penal condenatória acarretará a imediata exclusão do herdeiro ou legatário indigno, independentemente da sentença prevista no caput do art. 1.815 deste Código.    (Incluído pela Lei nº 14.661, de 2023)'

In [77]:
full_list = []
for art_list in contexto_df["Contexto"]:
    row_list = []
    for art_id in art_list:
        art_full = get_article(art_id.strip())
        row_list.append(art_full)
        if not art_full:
            print(art_id)
    full_list.append('\n'.join(row_list))

contexto_df['completo'] = full_list
contexto_df

,Contexto,OldContexto,completo
0,"[Art. 1214, Art. 1975, Art. 206, Art. 1974, 18...","Art 1214,Art 1975,Art 206,Art 1974,1826,Art 19...","Art. 1.214. O possuidor de boa-fé tem direito,..."
1,"[Art. 1880, Art. 1879, Art. 1867, Art. 1868]","Art 1880,Art 1879,Art 1867,Art 1868",Art. 1.880. O testamento particular pode ser e...
2,"[Art. 1858, Art. 1860, Art. 1880, Art. 1876, A...","Art 1858,Art 1860,Art 1880,Art 1876,Art 1859,A...","Art. 1.858. O testamento é ato personalíssimo,..."
3,"[Art. 1860, Art. 1861, Art. 1802, Art. 1900, A...","Art 1860,Art 1861,Art 1802,Art 1900,Art 1857,A...","Art. 1.860. Além dos incapazes, não podem test..."
4,"[Art. 1723, Art. 1521]","Art 1723,Art 1521",Art. 1.723. É reconhecida como entidade famili...
5,[Art. 276],Art 276,Art. 276. Se um dos devedores solidários falec...
6,"[Art. 1647, Art. 1580, Art. 1648, Art. 1651, A...","Art 1647,Art 1580,Art 1648,Art 1651,Art 1597,A...",Art. 1.647. Ressalvado o disposto no art. 1.64...
7,[Art. 270],Art 270,Art. 270. Se um dos credores solidários falece...
8,[Art. 1813],Art 1813,Art. 1.813. Quando o herdeiro prejudicar os se...
9,"[Art. 1860, Art. 1858, Art. 1857, Art. 1845, A...","Art 1860,Art 1858,Art 1857,Art 1845,Art 1786","Art. 1.860. Além dos incapazes, não podem test..."


In [78]:
df['contexto_artigos_full'] = contexto_df['completo']

In [79]:
def loop_fn(df, fn, kwarg=None):
    list_of_outputs = []
    for index, row in tqdm(df.iterrows(), total=len(df)):
        try:
            if kwarg:
                output = fn(row, kwarg)
            else:
                output = fn(row)
            print(output)
        except Exception as e:
            print(f"Error processing row {index}: {e}")
            output = "Error"
        list_of_outputs.append(output)
    return list_of_outputs

In [80]:
class BASELINE_JSON(BaseModel):
    analise: str = Field(description="Análise da questão, resposta e artigos de contexto.")
    resposta: str = Field(description="Resposta obtida pela análise dos trechos selecionados.")

output_parser = JsonOutputParser(pydantic_object=BASELINE_JSON)

In [81]:
prompt = """
Você é professor de Direito e ávido conhecedor do Código Civil Brasileiro.
Use seus conhecimentos em leis e códigos analisar atentamente os itens da questão.

Para te ajudar a responder foram levantados alguns artigos relevantes para a questão.
Em seguida responda corretamente a questão, sinalizando na resposta somente a letra da alternativa.

questão: {questão}
artigos: {artigos}

Gere somente as informações dos campos do JSON, sem introduções ou formalismos.
Instruções de saída: {format_instructions}
""".strip()

prompt_template = PromptTemplate(
    input_variables=["questão", "artigos"],
    template=prompt,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt_template | llama3_70b_temp0 | output_parser

In [82]:
def call_exp(sample):
    output = chain.invoke(
        {"questão": sample['pergunta'],
         "artigos": sample['contexto_artigos_full'],
         })
    return output

# test
out = call_exp(df.iloc[15])
out

{'analise': 'A questão aborda o direito de família, especificamente sobre os alimentos. Os artigos do Código Civil Brasileiro mencionados são relevantes para a análise da questão. O art. 1.696 estabelece que o direito à prestação de alimentos é recíproco entre pais e filhos, e extensivo a todos os ascendentes. O art. 1.694 estabelece que os alimentos devem ser fixados na proporção das necessidades do reclamante e dos recursos da pessoa obrigada. O art. 1.698 estabelece que se o parente que deve alimentos em primeiro lugar não estiver em condições de suportar totalmente o encargo, serão chamados a concorrer os de grau imediato. Já o art. 1.707 estabelece que o credor não pode renunciar o direito a alimentos.',
 'resposta': 'B'}

In [83]:
list_of_outputs = loop_fn(df, call_exp)

  3%|▎         | 1/37 [00:01<00:38,  1.08s/it]

{'analise': 'A questão apresenta um caso de herança, em que Diego, herdeiro aparente e de boa-fé, herdou um sítio de seu pai Euclides. Posteriormente, João Carlos, outro filho de Euclides, ingressou com ação de nulidade de inventário cumulada com petição de herança, provando sua paternidade. Diante disso, é necessário analisar os direitos de Diego em relação aos frutos e benfeitorias realizadas no sítio.', 'resposta': 'B'}


  5%|▌         | 2/37 [00:01<00:31,  1.12it/s]

{'analise': 'Análise das afirmativas: I - Verdadeiro, II - Falso, III - Verdadeiro, IV - Verdadeiro', 'resposta': 'B'}


  8%|▊         | 3/37 [00:02<00:31,  1.09it/s]

...

{'analise': 'A questão afirma que a legítima dos herdeiros necessários não pode ser incluída no testamento. Isso está correto, pois o art. 1.857, § 1o do Código Civil Brasileiro estabelece que a legítima dos herdeiros necessários não pode ser incluída no testamento.', 'resposta': 'C'}


 97%|█████████▋| 36/37 [06:11<00:11, 11.08s/it]

{'analise': 'A questão envolve a prescrição da petição de herança em uma ação de investigação de paternidade post mortem. De acordo com o art. 205 do Código Civil, a prescrição ocorre em 10 anos, quando a lei não fixou prazo menor. No caso, a abertura da sucessão ocorreu em 5/2/2010 e a petição inicial foi protocolada em 15/12/2022, portanto, não há prescrição. Além disso, o art. 1.822 do Código Civil estabelece que, decorridos 5 anos da abertura da sucessão, os bens arrecadados passarão ao domínio do Município ou do Distrito Federal, mas isso não se aplica ao caso, pois Renato se habilitou antes desse prazo.', 'resposta': 'A'}


100%|██████████| 37/37 [06:20<00:00, 10.28s/it]

{'analise': 'A questão apresenta uma situação hipotética em que João, nascido em Brasília - DF, viveu toda a sua vida em Penha - SC e morreu em Florianópolis - SC, deixando sua esposa Ana grávida. O Código Civil Brasileiro estabelece que os nascituros têm legitimidade para suceder, desde que concebidos no momento da abertura da sucessão (Art. 1.798).', 'resposta': 'C'}


In [84]:
exp3_cot_conttodos_ds2 = pd.DataFrame(list_of_outputs)
exp3_cot_conttodos_ds2.to_csv("exp3_cot_conttodos_ds2.csv", index=False)

In [89]:
exp3_cot_conttodos_ds2 = pd.read_csv("/content/exp3_cot_conttodos_ds2.csv")

In [90]:
exp3_cot_conttodos_ds2

,analise,resposta
0,"A questão apresenta um caso de herança, em que...",B
1,"Análise das afirmativas: I - Verdadeiro, II - ...",B
2,A questão apresenta uma situação hipotética em...,E
3,A questão apresenta uma situação hipotética em...,D
4,A questão apresenta uma situação complexa de r...,D
5,A questão apresenta uma situação em que João e...,C
6,A questão apresenta um caso de divórcio culpos...,C
7,A questão apresenta uma situação em que João e...,E
8,A questão apresenta um caso de renúncia à hera...,E
9,A questão apresenta quatro alternativas sobre ...,B


In [91]:
r_a = pd.DataFrame({
    'resposta_baseline': exp3_cot_conttodos_ds2['resposta'],
    'resposta': df['resposta'],
    'comparativo': exp3_cot_conttodos_ds2['resposta'] == df['resposta']})

r_a

,resposta_baseline,resposta,comparativo
0,B,C,False
1,B,B,True
2,E,E,True
3,D,C,False
4,D,C,False
5,C,E,False
6,C,E,False
7,E,E,True
8,E,D,False
9,B,B,True


In [92]:
r_a['comparativo'].value_counts()

comparativo
True     24
False    13
Name: count, dtype: int64

In [93]:
24/37

0.6486486486486487